In [11]:
!pip install --upgrade google-api-python-client
!pip install pandas_gbq
!pip install mpu
import sqlite3
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq
from scipy.spatial import distance
import numpy as np
import mpu

Requirement already up-to-date: google-api-python-client in c:\users\celherme1\anaconda3\lib\site-packages (1.7.8)


In [2]:
# Get a dataframe of all the cities at hand, along with their latitude and longitude
conn = sqlite3.connect('../cityDB.sqlite')
cities = pd.read_sql_query("SELECT cid, city, state, lat, lng  FROM cities;", conn)

# Get the coordinates for the cities
cities['coordinates'] = cities.apply(lambda x: (x['lat'], x['lng']), axis=1)

In [29]:
# Get a list of all the stations in the US from NOAA data publicly available on BigQuery
credentials = service_account.Credentials.from_service_account_file('bigquery_key.json')
stations_query = ("""SELECT *
FROM `bigquery-public-data.noaa_gsod.stations`
WHERE country = 'US'""")
stations = pandas_gbq.read_gbq(stations_query,
                               project_id="dva-destination-recommender",
                               credentials=credentials, dialect='standard')

# Remove stations with null values
stations.dropna(axis=0, inplace=True)

# Get the coordinates for the stations
stations['coordinates'] = stations.apply(lambda x: (x.lat, x.lon), axis=1)

In [4]:
# Get the monthly weather over the past 5 years for all stations
weather_query = ("""
SELECT stn, wban, mo as month, AVG(temp) as avg_temp, AVG(prcp) as avg_prcp, SUM(CAST(fog AS int64))/5 as foggy_days, 
SUM(CAST(rain_drizzle as int64))/5 as rainy_days, 
SUM(CAST(snow_ice_pellets as int64))/5 as snow_days, 
SUM(CAST(thunder as int64))/5 as stormy_days
FROM
(SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2018`
UNION ALL
SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2017`
UNION ALL
SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2016`
UNION ALL
SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2015`
UNION ALL
SELECT
  *
FROM 
 `bigquery-public-data.noaa_gsod.gsod2014`) a
 GROUP BY 1,2,3""")

weather = pandas_gbq.read_gbq(weather_query,
                               project_id="dva-destination-recommender",
                               credentials=credentials, dialect='standard')

In [30]:
# Only keep the stations for which we have weather info
stations_with_weather = weather.merge(stations, how='inner', left_on=['stn', 'wban'], right_on=['usaf', 'wban'])

In [68]:
def find_closest_station(city_coord, stations):
    """
    INPUTS:
    city_coord (tuple): a tuple with the coordinates for the city
    stations (Pandas DataFrame): a Pandas DataFrame with station information

    OUTPUT:
    ids (int): the station ID for the closest station to the city, both usaf and wban
    """
    # Initialize a dictionary to hold the distance with each station coordinates
    station_dict = {}
    
    for station in stations['coordinates'].unique().tolist():
        station_dict[station] = mpu.haversine_distance(city_coord, station)
        
    closest_station = sorted(station_dict.items(), key=lambda x: x[1])[0][0]
    
    usaf = stations.loc[stations['coordinates'] == closest_station, 'usaf']
    wban = stations.loc[stations['coordinates'] == closest_station, 'wban']

    ids = str(usaf) + '|' + str(wban)

    return ids

In [ ]:
# Find the closest station for each city in the dataframe
closest_stations = []
for row_num in range(len(cities)):
    city_coord = cities.iloc[row_num].coordinates
    closest_stations.append(find_closest_station(city_coord, stations_with_weather))

In [8]:
cities['closest_station'] = closest_stations

In [9]:
cities[['closest_station_usaf', 'closest_station_wban']] = cities['closest_station'].str.split('|',expand=True)

In [18]:
# Append weather information to the cities
city_monthly_weather = cities.merge(stations_with_weather, how='left', left_on=['closest_station_wban', 'closest_station_usaf'],
                                   right_on=['wban', 'stn'])

In [21]:
# Drop unecessary columns and columns with unsupported types in sqlite
city_monthly_weather.drop(labels=['usaf', 'name', 'country', 'state_y', 'coordinates_x', 
                                 'call', 'lat_y', 'lon', 'elev', 'begin', 'end', 'coordinates_y', 'stn', 'wban'], 
                          inplace=True, axis=1)

In [23]:
# Rename columns
city_monthly_weather.rename(axis=1, mapper={'state_x': 'state', 'lat_x': 'lat', 'lng': 'lon'}, inplace=True)

In [26]:
# Store to database
city_monthly_weather.to_sql('city_weather', conn, if_exists='replace')